# (Pseudo-) Online Feature Calculation

This notebook is used to receive data from the HoloLens 2. The data is chunked in 10 second pieces.

In [ ]:
from flask import Flask, request
import json
import requests
import os
import pandas as pd

## Read the data from the newly arrived csv-file and call the feature calculation

### Run FeaturesCalculation notebook to make its function accessible here

In [ ]:
%run -i FeatureCalculation.ipynb

## Calculate the features and save them as csv
See `FeaturesCalculation.ipynb`for more details.

In [ ]:
def calculate_features_for_10s_chunk(newdf):
    list_of_features = []
    newdf_valid = only_valid_data(newdf)
    if (len(newdf_valid) > 1):
        df_fixations = get_fixation_df(newdf_valid)
        features = calculate_fixation_features(df_fixations, 10)
        blinks = calculate_blink_features(newdf,10)
        directions = calculate_directions_of_list(df_fixations)
        density = calculate_fixation_density(newdf_valid, df_fixations)
        features.update(blinks)
        features.update(directions)
        features.update(density)

        # keep only the features the classifier expects
        sig_feats = get_significant_features()
        filtered = {k: features[k] for k in sig_feats if k in features}

        filtered["label"] = ""
        filtered["duration"] = "10"
        filtered["participant_id"] = "002"
        list_of_features.append(filtered)

        feature_file_path = save_as_csv(list_of_features, "001", './OnlineFeatureFiles/')
        return feature_file_path

In [ ]:
def csv_to_features(gaze_data_file_path):
    # continue only if the csv-file contains data
    if os.stat(gaze_data_file_path).st_size != 0:
        df = pd.read_csv(gaze_data_file_path)
        feature_file_path = calculate_features_for_10s_chunk(df)
        print(f"Feature calculation done for: {gaze_data_file_path}")
        print(f"Feature file path: {feature_file_path}")
        return feature_file_path
    else:
        print(f"File {gaze_data_file_path} is empty!")
        return ""
# csv_to_features("./HL2_DataCollection/2022_09_23-13_41_19-Alex01-Inspection02.csv")
# uncomment the line above to test the function with a local csv file

## Run SVM notebook to make its function accessible here
If you are not training on normalized data this make take a while.

In [ ]:
import joblib
import os
import numpy as np

MODEL_PATH = "./Models/extra_trees_classifier.joblib"
SCALER_PATH = "./Models/feature_scaler.joblib"

if os.path.exists(MODEL_PATH) and os.path.exists(SCALER_PATH):
    classifier = joblib.load(MODEL_PATH)
    scaler = joblib.load(SCALER_PATH)
    print("Loaded ExtraTrees model and scaler.")
else:
    raise FileNotFoundError(
        "Model or scaler not found. Run AnExtraTreesClassifierForHL2GazeFeatures.ipynb first to create './Models/extra_trees_classifier.joblib' and './Models/feature_scaler.joblib'."
    )

def get_significant_features():
    return [
        "freqDisPerSec",
        "meanFix",
        "maxFix",
        "varFix",
        "fixDensPerBB",
        "blinkRate",
        "meanDis",
        "minDis",
    ]

## Predict the class for the last arrived data chunk

In [ ]:
def normalize_values(df):
    # df: DataFrame with columns matching get_significant_features()
    sig_feats = get_significant_features()
    # enforce column order and presence
    missing = [c for c in sig_feats if c not in df.columns]
    if missing:
        raise ValueError(f"Cannot normalize — missing features: {missing}")
    X_ordered = df[sig_feats].copy()
    try:
        X_scaled = scaler.transform(X_ordered)  # use saved StandardScaler
    except Exception as e:
        raise RuntimeError(f"Scaler transform failed. Check scaler and feature order. Details: {e}")
    scaled_df = pd.DataFrame(X_scaled, columns=sig_feats, index=df.index)
    row_for_last_chunk = scaled_df.tail(1)
    print("Normalized Features:")
    display(row_for_last_chunk)
    return row_for_last_chunk

In [ ]:
row_counter = 0

def predict_class_for_last_chunk(feature_file_path, prob_threshold=0.5):
    global row_counter
    if os.stat(feature_file_path).st_size == 0:
        return

    df = pd.read_csv(feature_file_path)
    cur_number_of_rows = df.shape[0]

    # only continue if we have a new row
    if cur_number_of_rows <= row_counter:
        return

    last_row = df.tail(1)
    display(last_row)

    sig_feats = get_significant_features()
    missing = [f for f in sig_feats if f not in last_row.columns]
    if missing:
        print("Missing features in feature file:", missing)
        row_counter = cur_number_of_rows
        return

    X = last_row[sig_feats]
    X_norm = normalize_values(X)

    pred_class = classifier.predict(X_norm)[0]
    pred_proba = classifier.predict_proba(X_norm)[0]
    max_prob = float(np.max(pred_proba))

    print(f"Predicted: {pred_class} (confidence {max_prob:.3f})")

    if max_prob >= prob_threshold:
        send_activity(pred_class, max_prob)
    else:
        print("Confidence below threshold; not sent.")

    row_counter = cur_number_of_rows
            
# predict_class_for_last_chunk("FeatureFiles/feature_list_P09.csv")

## Send the recognized activity back to the HoloLens 2 

⚠️ Make sure to set the correct IP address for the HL2!

If you run the app via Holographic remoting, this is your PC's IP address.

In [ ]:
import urllib.parse
import requests
import random
import time

# Remember to change the IP address and port if needed!
# Start the HL2 app first, an input the url and port that you see in MR here
holo_url = "http://10.2.2.152:52739"

def send_activity(activity, probability):

    url = "{}/?activity={}&probability={}".format(str(holo_url), str(activity), str(probability))
    print(url)

    try:
        r = requests.get(url, timeout=120)
        print(r)
        if r.status_code == 200:
            print("Notified Hololens about activity {}".format(activity))
        else:
            print("Request {} failed with status code {}".format(url, r.status_code))
    except requests.ConnectionError as e:
        print("OOPS!! Connection Error. Make sure you are connected to Internet. Technical Details given below.\n")
        print(str(e))
    except requests.Timeout as e:
        print("OOPS!! Timeout Error")
        print(str(e))
    except requests.RequestException as e:
        print("OOPS!! General Error")
        print(str(e))

    else:
        return

def create_and_send_test_data():
    activities = ["reading", "writing", "searching", "inspecting"]
    a = random.randint(0, 3)
    activity = activities[a]
    confidence = random.uniform(0, 1)
    print(f"{activity}: {confidence}")
    send_activity(activity, confidence)


def sender():
    start_time = time.time()
    interval = 5
    for i in range(20):
        time.sleep(start_time + i * interval - time.time())
        create_and_send_test_data()
        print("sent data")

def send_this_desktop_ip_to_holo(desktop_ip, port):
    
    desktop_ip = urllib.parse.quote(desktop_ip)

    url = "{}/?desktopip={}&port={}".format(str(holo_url), str(desktop_ip), str(port))
    print(url)

    try:
        r = requests.get(url, timeout=60)
        print(r)
        if r.status_code == 200:
            print("Notified Hololens about IP address {}".format(desktop_ip))
        else:
            print("Request {} failed with status code {}".format(url, r.status_code))
    except requests.ConnectionError as e:
        print("OOPS!! Connection Error. Make sure you are connected to Internet. Technical Details given below.\n")
        print(str(e))
    except requests.Timeout as e:
        print("OOPS!! Timeout Error")
        print(str(e))
    except requests.RequestException as e:
        print("OOPS!! General Error")
        print(str(e))

    else:
        return
    
    
# if __name__ == '__main__':
    # sender()


## Send this desktop's IP address to the HL2
This enables the HL2 to send the gaze data correctly to the computer where this script is running.
Make sure to enter your computer's IP-address and a free port.

In [ ]:
import socket

def find_free_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))  # 0 means to select an arbitrary unused port
        return s.getsockname()[1]

free_port = find_free_port()
print(f"Free port found: {free_port}")

In [ ]:
ipaddress = "10.2.2.152"
# ipaddress = "localhost"
# ipaddress = "130.82.27.178"
send_this_desktop_ip_to_holo(ipaddress, free_port)

## Run a simple Flask server that receives the raw gaze data from the HL2
Make sure to enter your computer's IP-address and a suitable port in the last line. \
Note: `0.0.0.0` as IP address lets the server listen on all its IP addresses, usually 127.0.0.1 and the public IP address.

In [ ]:
from flask_executor import Executor

import logging
logger = logging.getLogger('waitress')
logger.setLevel(logging.DEBUG)

app = Flask(__name__)
executor = Executor(app)

@app.route('/', methods=['POST', 'PUT'])
def result():
    new_csv = request.files["gazedata"].read()
    filename = request.form["filename"]
    print(f"filename: {filename}")
    filepath = os.path.join("./OnlineGazeDataChunks/", filename)
    print(f"filepath: {filepath}")
    outF = open(filepath, "wb")
    outF.write(new_csv)
    executor.submit(start_feature_calculation,filepath)
    return 'Received !'  

# '''
def start_feature_calculation(filepath):
    print("start fc for: ", filepath)
    if os.stat(filepath).st_size != 0:
        print(f"File {filepath} is not empty, start feature calculation")
        feature_file_path = csv_to_features(filepath)
        if os.stat(feature_file_path).st_size != 0:
            print("start prediction for: ", feature_file_path)
            predict_class_for_last_chunk(feature_file_path)
        else:
            print(f"File {feature_file_path} is empty!")
    else:
        print(f"File {filepath} is empty!")
# '''

if __name__ == '__main__':
    from waitress import serve
    serve(app, listen=f"0.0.0.0:{free_port}")
    # serve(app, listen="10.2.2.152:5555")
